In [1]:
import pandas as pd
import numpy as np
import math
import json
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [7]:
splitter=transcript.time >= 24*7*3
test_transcript=transcript[splitter].copy()
transcript=transcript[~splitter]

In [9]:
profile=profile.set_index('id')

portfolio=portfolio.set_index('id')
portfolio.index.rename('offer_id', inplace=True)

dummies=portfolio.channels.apply(lambda x: pd.Series(len(x)*[1], x)).fillna(0)
portfolio=pd.concat([portfolio.drop('channels', axis=1), dummies], axis=1)
portfolio.sort_values(['offer_type','reward','difficulty'])

,reward,difficulty,duration,offer_type,email,mobile,social,web
offer_id,,,,,,,,
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0
f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1.0,1.0,1.0,1.0
ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,bogo,1.0,1.0,1.0,0.0
4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,bogo,1.0,1.0,1.0,1.0
fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,1.0,1.0,1.0,1.0
2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1.0,1.0,0.0,1.0
2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,discount,1.0,1.0,1.0,1.0
0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1.0,0.0,0.0,1.0
3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1.0,1.0,0.0,1.0


In [10]:
offers_made=transcript[transcript.event=='offer received'].copy()
offers_made['offer_id']=offers_made.value.apply(lambda x: x['offer id'])
offers_made.drop(['event', 'value'], axis=1, inplace=True)
offers_recieved=offers_made.merge(portfolio, left_on='offer_id', right_index=True)
offers_made=offers_made.groupby(['offer_id','time']).count().person
offers_made.name='count'


In [11]:
offers_made=offers_made.reset_index().merge(portfolio[['duration', 'reward']], left_on='offer_id', right_index=True)
offers_recieved=offers_recieved.merge(portfolio.duration, left_on='offer_id', right_index=True)

In [12]:
offers_made['end']=offers_made.time+offers_made.duration*24
offers_recieved['end']=offers_recieved.time+offers_made.duration*24

In [13]:
offers_recieved.head()

,person,time,offer_id,reward,difficulty,duration_x,offer_type,email,mobile,social,web,duration_y,end
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,240.0
18,ebe7ef46ea6f4963a7dd49f501b26779,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,120.0
21,f082d80f0aac47a99173ba8ef8fc1909,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,72.0
28,c0d210398dee4a0895b24444a5fcd1d2,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,168.0
30,57dd18ec5ddc46828afb81ec5977bef2,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1.0,1.0,0.0,1.0,7,168.0


In [14]:
offers_made.head()

,offer_id,time,count,duration,reward,end
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1297,10,5,240
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,168,1245,10,5,408
2,0b1e1539f2cc45b7b9fa7c272da2e1d7,336,1322,10,5,576
3,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,1269,10,5,648
4,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1221,7,3,168


In [15]:
offers_completed=transcript[transcript.event=='offer completed'].copy()
offers_completed['offer_id']=offers_completed.value.apply(lambda x: x['offer_id'])
offers_completed.drop(['value','event'],axis=1, inplace=True)
offers_completed=offers_completed.merge(portfolio.reward, left_on='offer_id', right_index=True)

offers_completed.head()

,person,time,offer_id,reward
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5,2
12764,73ffefd41e9a4ca3ab26b2b3697c6eb7,0,2906b810c7d4411798c6938adc9daaa5,2
12786,24115a61df25473e84a8a03f3c98de1a,0,2906b810c7d4411798c6938adc9daaa5,2
12826,2481f1fcfbcb4b288e5a03af02d95373,0,2906b810c7d4411798c6938adc9daaa5,2
13183,a45b69f1c8554ae7af83e74426ca437a,0,2906b810c7d4411798c6938adc9daaa5,2


In [16]:
transactions=transcript[transcript.event=='transaction'].copy()
transactions.value=transactions.value.apply(lambda x: x['amount'])
transactions.set_index('person', inplace=True)
transactions.head()

,event,value,time
person,,,
02c083884c7d45b39cc68e1314fec56c,transaction,0.83,0
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,34.56,0
54890f68699049c2a04d415abc25e717,transaction,13.23,0
b2f1cd155b864803ad8334cdf13c4bd2,transaction,19.51,0
fe97aa22dd3e48c8b143116a8403dd52,transaction,18.97,0


In [17]:
def pull_relevant(offer,start,end):
    is_not_null=offers_recieved.set_index('person').apply(
        lambda row: start <= row.time <= end or start <= row.end <= end, axis=1).groupby('person').any()
    not_null=profile.loc[is_not_null.index][is_not_null]
    null=profile.drop(not_null.index)
    offered=offers_recieved.set_index(['time','offer_id']).sort_index().loc[(start,offer),'person'].values
    offered=not_null.loc[offered]
    
    period_transactions=transactions[(start <= transactions.time) & (transactions.time <= end)].value
    
    merge_group = lambda L,R: L.merge(R, left_index=True, right_index=True, how='left').groupby(level=0)
    collate =  lambda X: pd.DataFrame([X.count(),X.sum(),X.mean()], index=['count', 'total', 'mean']).transpose().fillna(0)
    
    null_transactions=merge_group(null,period_transactions).value
    null=null.merge(collate(null_transactions), left_index=True, right_index=True)
    #null.join(null,collate(null_transactions))
    
    offered_transactions=merge_group(offered,period_transactions).value
    offered=offered.merge(collate(offered_transactions), left_index=True, right_index=True)
    
    return null,offered
    
                        

In [18]:
from scipy.stats import ttest_ind

In [19]:
offer, start, end = '0b1e1539f2cc45b7b9fa7c272da2e1d7', 0, 240

In [37]:
def t_test(offer,start,end, cost):
    null,offered=pull_relevant(offer,start,end)
    cols=['age', 'income']
    p_dict={('p_'+col):ttest_ind(null[col].dropna(), offered[col].dropna()).pvalue for col in cols}
    
    
    def t_test_1_sided(a,b,c):
        
        p=ttest_ind(a.dropna()-c, b.dropna(), equal_var=False).pvalue/2
        if a.mean()-c < b.mean():
            p=1-p
        return -np.log(p)
    
    p_dict['p_total-cost']=t_test_1_sided(offered['total'], null['total'], cost)
    p_dict['total-cost']=offered['total'].mean()-null['total'].mean()-cost
    
    for col in ['total','mean', 'count']:
        p_dict['p_'+col]=t_test_1_sided(offered[col],null[col],0)
        p_dict['offered_'+col]=offered[col].mean()
        p_dict['null_'+col]=null[col].mean()
        
    
    p_series=pd.Series(p_dict)
    
    return p_series 

In [38]:
t_test(offer,start,end, 5/2)

p_age             0.637936
p_income          0.151557
p_total-cost     17.774222
total-cost       11.386614
p_total          25.257765
offered_total    29.393778
null_total       15.507164
p_mean            6.989201
offered_mean     11.080959
null_mean         8.311168
p_count          80.160743
offered_count     2.087124
null_count        1.268877
dtype: float64

In [39]:
A=offers_made.apply(lambda row: t_test(row.offer_id,row.time,row.end, row.reward), axis=1)
A.index=offers_made.set_index(['offer_id', 'time']).index

In [41]:
res_df=pd.concat([portfolio[['offer_type', 'reward', 'difficulty', 'duration']], 
            A.reset_index().groupby('offer_id').mean().drop(['time', 'p_age', 'p_income'], axis=1)], 
          axis=1).sort_values('offer_type')
res_df
##Mean is not a good way of combining - use a chi^2 test or recompute

,offer_type,reward,difficulty,duration,p_total-cost,total-cost,p_total,offered_total,null_total,p_mean,offered_mean,null_mean,p_count,offered_count,null_count
ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,8.705513,4.520587,43.859167,27.036844,12.516258,6.618037,10.546398,8.162868,186.579516,2.275956,0.998897
4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,5.572474,2.498635,39.586282,22.491025,9.992390,12.768787,10.055979,6.978783,189.957594,1.824272,0.800128
9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,5.732271,3.149104,18.325936,20.665362,12.516258,2.713333,9.311866,8.162868,74.465127,1.633541,0.998897
f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,16.627398,7.824001,38.914390,22.816391,9.992390,11.744487,10.613382,6.978783,178.126037,1.765020,0.800128
0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,20,10,8.069708,5.475095,19.669389,25.574687,15.099592,4.181861,10.769954,8.854724,55.505678,1.863804,1.227642
2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3,7,7,39.126233,13.056311,56.804214,28.572569,12.516258,7.712018,10.575627,8.162868,206.841012,2.340822,0.998897
fafdcd668e3743c1bb461111dcafc2a4,discount,2,10,10,54.680809,23.218595,63.419642,40.318187,15.099592,8.649599,11.955825,8.854724,236.489346,3.130536,1.227642
2906b810c7d4411798c6938adc9daaa5,discount,2,10,7,14.129658,7.306072,20.730521,21.822330,12.516258,4.657922,9.953847,8.162868,76.006308,1.645767,0.998897
3f207df678b143eea3cee63160fa8bed,informational,0,0,4,12.836185,3.763400,12.836185,12.513354,8.749955,3.871755,7.374878,6.374291,40.843232,1.004417,0.699902
5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,26.795898,5.425108,26.795898,12.463181,7.038073,11.206564,7.506624,5.363972,104.769207,1.079949,0.556157


In [42]:
res_df['total-cost']/res_df.duration

ae264e3637204a6fb9bb56bc8210ddfd    0.645798
4d5c57ea9a6940dd891ad53e9dbe8da0    0.499727
9b98b8c7a33c4b65b9aebfe6a799e6d9    0.449872
f19421c1d4aa40978ebb69ca19b0e20d    1.564800
0b1e1539f2cc45b7b9fa7c272da2e1d7    0.547509
2298d6c36e964ae4a3e7e9706d1fb8c2    1.865187
fafdcd668e3743c1bb461111dcafc2a4    2.321859
2906b810c7d4411798c6938adc9daaa5    1.043725
3f207df678b143eea3cee63160fa8bed    0.940850
5a8bc65990b245e5a138643cd4eb9837    1.808369
dtype: float64

All the offers are effective against this measure. Roughly speaking it looks like the 2,10,10 discount performs best, but comparing the offers like this is on shaky statistical footing. There may be other factors impacting performance, e.g. people might be more likely to buy coffee on a monday, so offers which include a (or multiple) monday might get a boost. 

In [43]:
A.loc['fafdcd668e3743c1bb461111dcafc2a4']['p_total-cost']

time
0      82.875572
168    63.867601
336    54.096205
408    17.883859
Name: p_total-cost, dtype: float64